In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rank_bm25
!pip install transformers
!pip install sentencepiece
!pip install wget
!pip install sense2vec
!pip install thinc
!python -m spacy download en_core_web_md 

In [ ]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import pandas as pd
import numpy as np
import json
import requests
import re
import nltk
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import STOPWORDS
import string
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from tqdm import tqdm
from rank_bm25 import BM25Okapi
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import sys

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def parse_xml(path):
  answer_list = []
  xmldoc = minidom.parse(path)
  itemlist = xmldoc.getElementsByTagName('topics')
  topic_list = itemlist[0].getElementsByTagName('topic')
  for topic in topic_list:
    tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue,topic.getElementsByTagName('objects')[0].firstChild.nodeValue))
    answer_list.append(tuple_for_add)
  parsed=pd.DataFrame(answer_list, columns=["Number","Title","Objects"])
  return parsed

In [ ]:
topics=parse_xml("/content/drive/MyDrive/Touche/topics-task2.xml")
topics.head()

,Number,Title,Objects
0,2,"Which is better, a laptop or a desktop?","laptop, desktop"
1,3,"Which is better, Canon or Nikon?","Canon, Nikon"
2,8,What are the advantages and disadvantages of P...,"PHP, Python"
3,9,Why is Linux better than Windows?,"Linux, Windows"
4,12,Train or plane? Which is the better choice?,"Train, plane"


In [ ]:
jsonObj = pd.read_json(path_or_buf='/content/drive/MyDrive/Touche/touche-task2-passages-version-002.jsonl', lines=True)
jsonObj.head()

# Chat Noir

In [ ]:
#extract the uuid from the chat noir url
def regex_parser(x):
  return(re.search(".*=(.*)&index.*",x)[1])
jsonObj["uuid"]=jsonObj["chatNoirUrl"].apply(regex_parser)
jsonObj.head()

,id,contents,chatNoirUrl,uuid
0,clueweb12-0000tw-14-21168___1,"Shuga: Love, Sex, Money MTV Shuga Home Swag Bl...",https://chatnoir.eu/cache?uuid=f338e91e-a3e9-5...,f338e91e-a3e9-52ec-bb4c-cac853f4df55
1,clueweb12-0000tw-14-21168___2,We LOVE sending #TeamShuga the exclusives. Ban...,https://chatnoir.eu/cache?uuid=f338e91e-a3e9-5...,f338e91e-a3e9-52ec-bb4c-cac853f4df55
2,clueweb12-0000tw-14-21168___3,Now take note.. because you will be seeing a w...,https://chatnoir.eu/cache?uuid=f338e91e-a3e9-5...,f338e91e-a3e9-52ec-bb4c-cac853f4df55
3,clueweb12-0000tw-22-19226___1,Sex and love: The modern matchmakers | The Eco...,https://chatnoir.eu/cache?uuid=2bf4b08d-2f65-5...,2bf4b08d-2f65-5741-80b2-0e6ec18d5505
4,clueweb12-0000tw-22-19226___2,But have they? Feb 11th 2012 | from the print ...,https://chatnoir.eu/cache?uuid=2bf4b08d-2f65-5...,2bf4b08d-2f65-5741-80b2-0e6ec18d5505


In [ ]:
#chatnoir api intialization
chatnoir = "https://www.chatnoir.eu/api/v1/_search"
attr = {"apikey": "7dd15626-53aa-46c6-bd34-b2feaa2d9d81",
        "query": "hello world",
        "index": "cw12",
        "pretty": True,
        "size":100}

In [ ]:
#search for a topic using chat noir api
for x in topics["Title"][0:1]:
    attr["query"] = x
    print(x)
    response = requests.post(chatnoir, data = attr)
    res = pd.json_normalize(response.json()["results"])
    doc_url = "https://www.chatnoir.eu/cache?uuid="+res['uuid']+"&index=cw12&raw&plain"

Which is better, a laptop or a desktop?


In [ ]:
#cross referncing the retrieved documents with the given dataset
for x in res['uuid']:
  print(x)
  print(x in jsonObj["uuid"].values)

# BM25

In [ ]:
#functions for preprocessing
special_characters=string.punctuation
porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()


def spl_chars_removal(lst):
  lst1=list()
  for element in lst:
    str=""
    str="".join([i for i in element if i not in special_characters])
    lst1.append(str)
  return lst1

def stopwords_removal_gensim_custom(lst):
  tokens_without_sw = [word for word in lst if not word in STOPWORDS]
  return tokens_without_sw

def stemming(lst):
  stem_text = [porter_stemmer.stem(word) for word in lst]
  return stem_text

def lemmatizer(lst):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in lst]
  return lemm_text

In [ ]:
#preprocessing the documents (not using apply function due to memory error)
preprocessed_documents=[]
for i in tqdm(jsonObj["contents"]):
  i=i.lower()
  z = i.split(" ")
  z=spl_chars_removal(z)
  #print(z)
  z=stopwords_removal_gensim_custom(z)
  #print(z)
  z=lemmatizer(z)
  z=' '.join(z)
  preprocessed_documents.append(z)
  #print(z)


100%|██████████| 868655/868655 [08:27<00:00, 1711.77it/s]


In [ ]:
tokenized_corpus = [doc.split(" ") for doc in preprocessed_documents]

In [ ]:
#preprocessing the queries
preprocessed_queries=[]
for i in tqdm(topics["Title"]):
  i=i.lower()
  z = i.split(" ")
  z=spl_chars_removal(z)
  #print(z)
  #z=stopwords_removal_gensim_custom(z)
  #print(z)
  z=lemmatizer(z)
  z=' '.join(z)
  preprocessed_queries.append(z)
  #print(z)

100%|██████████| 50/50 [00:00<00:00, 9290.94it/s]


In [ ]:
tokenized_query=[doc.split(" ") for doc in preprocessed_queries]
tokenized_query[0]

['which', 'is', 'better', 'a', 'laptop', 'or', 'a', 'desktop']

In [ ]:
#trainiing the bm25 models on our corpus
bm25 = BM25Okapi(tokenized_corpus)
bm25 =BM25L(tokenized_corpus)

In [ ]:
#fetching the top 1000 documents for a query
top_n=bm25.get_top_n(tokenized_query[0],preprocessed_documents, n=1000)
doc_scores = bm25.get_scores(tokenized_query[0])


# Predicting Queries from Passages:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('castorini/doc2query-t5-base-msmarco')
model = T5ForConditionalGeneration.from_pretrained('castorini/doc2query-t5-base-msmarco')
model.to(device)

In [ ]:
doc_text=jsonObj["contents"][0]
input_ids = tokenizer.encode(doc_text, return_tensors='pt').to(device)
outputs = model.generate(
    input_ids=input_ids,
    max_length=64,
    do_sample=True,
    top_k=10,
    num_return_sequences=3)

In [ ]:
for i in range(10):
    print(f'sample {i + 1}: {tokenizer.decode(outputs[i], skip_special_tokens=True)}')

#query expansion

In [ ]:
import logging
import tarfile
import nltk
import wget as wget
from nltk.corpus import wordnet

# from typing import List
import spacy

# en_core_web_md
import string
from tqdm import tqdm
from spacy.lang.en.stop_words import STOP_WORDS
import random
import numpy as np

# packages to find similar words by wordembedding and sense2vec
from sense2vec import Sense2VecComponent
from sense2vec import Sense2Vec  # for standalone
import pandas as pd
from pathlib import Path

random.seed(10)
# !/usr/bin/python


class QueryExpansion:
    def __init__(self, queries: pd.DataFrame, top_syns: int = 5, top_similar: int = 2):
        self.df_queries = queries

        self.nlp = spacy.load("en_core_web_md")
        self.top_syns = top_syns


        path = "/content/drive/MyDrive/Touche/s2v_old"
        self.sense = self.nlp.add_pipe("sense2vec").from_disk(path)
        self.standalone_s2v = Sense2Vec().from_disk(path)  # it is not dubplicated

        self.tags = ["CD", "JJ", "RB", "NN", "NNS", "NNP", "NNPS", "VB"]
        self.pos_tags = ["PROPN", "VERB", "NOUN", "NUM"]

        self.top_similar = top_similar

    # Query Expansion
    def expansion(
        self,
        relation: bool = False,
        synonyms: bool = False,
        sensevec: bool = False,
        embedded: bool = False,
    ):

        if relation:
            result = []
            result = [
                *result,
                *self.get_comparation_superlation_nouns_from_original_data(),
            ]
            self.df_queries = pd.concat(
                [
                    self.df_queries,
                    pd.DataFrame(result, columns=["TopicID", "topic", "query", "tag"]),
                ]
            )

        if synonyms:
            result = []
            result = [*result, *self.synonyms()]
            self.df_queries = pd.concat(
                [
                    self.df_queries,
                    pd.DataFrame(result, columns=["TopicID", "topic", "query", "tag"]),
                ]
            )

        if sensevec:
            result = []
            result = [*result, *self.similarwords_sensevec()]
            self.df_queries = pd.concat(
                [
                    self.df_queries,
                    pd.DataFrame(result, columns=["TopicID", "topic", "query", "tag"]),
                ]
            )
        if embedded:
            result = []
            result = [*result, *self.similarwords_wordembedding()]
            self.df_queries = pd.concat(
                [
                    self.df_queries,
                    pd.DataFrame(result, columns=["TopicID", "topic", "query", "tag"]),
                ]
            )

        return self.df_queries

    def similarwords_replace(self, query, similar_words):
        import copy

        expanded_queries = []
        for word, similar_words in similar_words.items():
            for similar_word in similar_words:
                new_query = copy.deepcopy(query).replace(word, similar_word)
                expanded_queries.append(new_query)
        return expanded_queries

    def similarwords_sensevec(self):
        result = []

        for original_query in tqdm(
            list(self.df_queries["topic"].unique()), desc="Sensevec progress"
        ):

            original_topicid = self.df_queries[
                self.df_queries["topic"] == original_query
            ].iloc[0]["TopicID"]

            doc = self.nlp(original_query)
            expanded_queries = []
            similar_words = {}
            for token in doc:
                top_similar_words = []
                if token.tag_ in self.tags or token.pos_ in self.pos_tags:
                    if token.lemma_ not in STOP_WORDS or token.text not in STOP_WORDS:
                        try:
                            for e in token._.s2v_most_similar(self.top_similar):
                                word = e[0][
                                    0
                                ].strip()  # get only word from ((word, tag), proba)
                                if (word != token.text) and (
                                    self.nlp(word)[0].lemma_ != token.lemma_
                                ):
                                    top_similar_words.append(word)
                        except ValueError as err:
                            print(err)
                            for ent in doc.ents:
                                if ent.text == token.text:
                                    try:
                                        for e in ent._.s2v_most_similar(
                                            self.top_similar
                                        ):
                                            word = e[0][
                                                0
                                            ].strip()  # get only word from ((word, tag), proba)
                                            if (word != token.text) and (
                                                self.nlp(word)[0].lemma_ != token.lemma_
                                            ):
                                                top_similar_words.append(word)

                                    except Exception as err:
                                        print(err)
                                        query = token._.s2v_other_senses[
                                            0
                                        ]  # get first similar words by entity_tag
                                        for e in self.standalone_s2v.most_similar(
                                            query, n=self.top_similar
                                        ):
                                            word = (
                                                e[0].split("|")[0].strip()
                                            )  # get only word from (word|tag, proba)
                                            if (word != token.text) and (
                                                self.nlp(word)[0].lemma_ != token.lemma_
                                            ):
                                                top_similar_words.append(word)

                        similar_words[token.text] = list(set(top_similar_words))
            expanded_queries = self.similarwords_replace(original_query, similar_words)
            i = 1
            for new_query in expanded_queries:
                result.append(
                    [original_topicid, original_query, new_query, "sensevec_" + str(i)]
                )
                i = i + 1
        return result

    def remove_punc(self, query: str):
        table = str.maketrans(dict.fromkeys(string.punctuation))
        title = query.translate(table)
        return str(title)

    def synonyms(self):
        result = []

        for query in tqdm(
            list(self.df_queries["topic"].unique()), desc="Synonyms progress"
        ):
            original_topicid = self.df_queries[self.df_queries["topic"] == query].iloc[
                0
            ]["TopicID"]

            new_title = self.remove_punc(query)
            syn_pro_title = list()
            temp = new_title
            new_title = self.nlp(new_title)
            for token in new_title:
                syn_token = self.find_syns_word(token)
                syn_pro_title.extend(
                    [syn for syn in list(set(syn_token)) if syn != str(token.text)]
                )  # distinct and remove the same words
            # print(syn_pro_title)
            # synonyms_by_titles.writelines(" ".join(list(set(syn_pro_title))) + "\n")
            # ToDo temp(org) + syns or only syns?
            result.append(
                [
                    original_topicid,
                    query,
                    temp + " " + " ".join(list(set(syn_pro_title))),
                    "syns",
                ]
            )
        return result

    def find_syns_word(self, token: str):
        syn_token = []
        if token.pos_ == "NOUN":
            # ToDo automate wordnet install
            for synset in wordnet.synsets(token.lemma_):
                for lemma in synset.lemmas()[: self.top_syns]:  # top 5 synonyms
                    if (
                        "_" not in lemma.name()
                    ):  # not include the words with _ ex: basketball_game
                        syn_token.append(lemma.name())
                    else:
                        for w in lemma.name().split("_"):
                            if w not in STOP_WORDS:
                                syn_token.append(
                                    w
                                )  # add words with _ to two words ex. laptop_computer -> laptop and computer
        syn_token = [w for w in syn_token if w != " " and len(w) != 0]

        # ToDo remove random influence
        if len(syn_token) > 5:
            return random.sample(
                syn_token, k=5
            )  # after top 5 synonyms + splited words -> long titles -> reducing the syns random with 5
        else:
            return syn_token

    def get_comparation_superlation_nouns_from_original_data(self):
        result = []

        for query in tqdm(
            list(self.df_queries["topic"].unique()), desc="Relation progress"
        ):
            original_topicid = self.df_queries[self.df_queries["topic"] == query].iloc[
                0
            ]["TopicID"]

            nouns_as_string = []
            doc = self.nlp(query)
            annotations = [
                "CC",
                "CD",
                "JJ",
                "JJR",
                "JJS",
                "RB",
                "RBR",
                "RBS",
                "NN",
                "NNS",
                "NNP",
                "NNPS",
                "VB",
            ]
            for token in doc:
                if token.tag_ in annotations:
                    nouns_as_string.append(token.text)
            result.append(
                [original_topicid, query, " ".join(nouns_as_string), "annotation"]
            )
        return result

    def similarwords_wordembedding(self):
        result = []
        for query in tqdm(
            list(self.df_queries["topic"].unique()), desc="Embeddings progress"
        ):
            original_topicid = self.df_queries[self.df_queries["topic"] == query].iloc[
                0
            ]["TopicID"]

            doc = self.nlp(query)
            # expanded queries
            expanded_queries = []
            similar_words = {}
            for token in doc:
                top_similar_words = []
                if token.tag_ in self.tags or token.pos_ in self.pos_tags:
                    if token.lemma_ not in STOP_WORDS or token.text not in STOP_WORDS:
                        try:
                            word = token.lemma_

                            try:
                                key = self.nlp.vocab.strings[word]
                                vector = self.nlp.vocab.vectors[key]
                                vector_asarray = np.asarray([vector])
                                ms = self.nlp.vocab.vectors.most_similar(
                                    vector_asarray, n=self.top_similar
                                )
                                # print(ms)
                                similar_embedding = [
                                    self.nlp.vocab.strings[w] for w in ms[0][0]
                                ]  # get only text from most_similar
                                # checking again if similar words are the same word
                                for word in similar_embedding:
                                    if (word != token.text) and (
                                        self.nlp(word)[0].lemma_ != token.lemma_
                                    ):
                                        top_similar_words.append(
                                            self.nlp(word)[0].lemma_.lower()
                                        )
                            except KeyError as err:
                                print(err)
                        except ValueError as err:
                            print(err)

                        similar_words[token.text] = list(
                            set(top_similar_words)
                        )  # only unique words

            # replace with similar words for new queries.
            expanded_queries = self.similarwords_replace(query, similar_words)

            i = 1
            for new_query in expanded_queries:
                result.append(
                    [original_topicid, query, new_query, "embedded_" + str(i)]
                )
                i = i + 1
        return result


# if __name__ == "__main__":

#     topics = [
#         "What is the difference between sex and love?",
#         "Which is the highest mountain in the world?",
#         "Which is better, a laptop or a desktop?",
#     ]

#     topics = [
#         "What is the difference between sex and love?",
#         "Which is better, a laptop or a desktop?",
#         "Which is better, Canon or Nikon?",
#         "What are the best dish detergents?",
#         "What are the best cities to live in?",
#         "What is the longest river in the U.S.?",
#         "Which is healthiest: coffee, green tea or black tea and why?",
#         "What are the advantages and disadvantages of PHP over Python and vice versa?",
#         "Why is Linux better than Windows?",
#         "How to sleep better?",
#         "Should I buy an LCD TV or a plasma TV?",
#         "Train or plane? Which is the better choice?",
#         "What is the highest mountain on Earth?",
#         "Should one prefer Chinese medicine or Western medicine?",
#         "What are the best washing machine brands?",
#         "Should I buy or rent?",
#         "Do you prefer cats or dogs, and why?",
#         "What is the better way to grill outdoors: gas or charcoal?",
#         "Which is better, MAC or PC?",
#         "What is better: to use a brush or a sponge?",
#         "Which is better, Linux or Microsoft?",
#         "Which is better, Pepsi or Coke?",
#         "What is better, Google search or Yahoo search?",
#         "Which one is better, Netflix or Blockbuster?",
#         "Which browser is better, Internet Explorer or Firefox?",
#         "Which is a better vehicle: BMW or Audi?",
#         "Which one is better, an electric stove or a gas stove?",
#         "What planes are best, Boeing or Airbus?",
#         "Which is better, Disneyland or Disney World?",
#         "Should I buy an Xbox or a PlayStation?",
#         "Which has more caffeine, coffee or tea?",
#         "Which is better, LED or LCD Reception Displays?",
#         "What is better: ASP or PHP?",
#         "What is better for the environment, a real or a fake Christmas tree?",
#         "Do you prefer tampons or pads?",
#         "What IDE is better for Java: NetBeans or Eclipse?",
#         "Is OpenGL better than Direct3D in terms of portability to different platforms?",
#         "What are the differences between MySQL and PostgreSQL in performance?",
#         "Is Java code more readable than code written in Scala?",
#         "Which operating system has better performance: Windows 7 or Windows 8?",
#         "Which smartphone has a better battery life: Xperia or iPhone?",
#         "Which four wheel truck is better: Ford or Toyota?",
#         "Should I prefer a Leica camera over Nikon for portrait photographs?",
#         "Which company has a larger capitalization: Apple or Microsoft?",
#         "Which laptop has a better durability: HP or Dell?",
#         "Which beverage has more calories per glass: beer or cider?",
#         "Is admission rate in Stanford higher than that of MIT?",
#         "Is pasta healthier than pizza?",
#         "Which city is more expensive to live in: San Francisco or New York?",
#         "Whose salary is higher: basketball or soccer players?",
#     ]

#     df = pd.DataFrame(
#         list(zip(range(0,len(topics)),topics, topics, len(topics) * ["original"])),
#         columns=["TopicID","topic", "query", "tag"],
#     )
#     print(df)

#     pd.set_option("display.max_columns", 4)
#     expansion = QueryExpansion(df)
#     df_relation = expansion.expansion(
#         relation=True, synonyms=True, sensevec=True, embedded=True
#     )

#     for q in topics:
#         print(q)
#         print(df_relation[df_relation["topic"] == q])
#         print(100 * "=")

In [ ]:
df = pd.DataFrame(
        list(zip(range(0,len(topics)),topics["Objects"], topics["Title"], len(topics) * ["original"])),
        columns=["TopicID","query", "topic", "tag"],)

In [ ]:
expansion = QueryExpansion(df)
df_relation = expansion.expansion(relation=True, synonyms=True, sensevec=True, embedded=True)

In [ ]:
df_relation

,TopicID,query,topic,tag
0,0,"laptop, desktop","Which is better, a laptop or a desktop?",original
1,1,"Canon, Nikon","Which is better, Canon or Nikon?",original
2,2,"PHP, Python",What are the advantages and disadvantages of P...,original
3,3,"Linux, Windows",Why is Linux better than Windows?,original
4,4,"Train, plane",Train or plane? Which is the better choice?,original
...,...,...,...,...
232,48,"Which is more environmentally friendly, a hybr...","Which is more environmentally friendly, a hybr...",embedded_8
233,49,Should I learn pyside or R for data analysis?,Should I learn Python or R for data analysis?,embedded_1
234,49,Should I learn Python or ¤ for data analysis?,Should I learn Python or R for data analysis?,embedded_2
235,49,Should I learn Python or R for dataset analysis?,Should I learn Python or R for data analysis?,embedded_3
